In [ ]:
!pip install kaggle-environments -U > /dev/null 2>&1
!cp -r ../input/lux-ai-2021/* .

todo：需要把所有的dest判断都与in——city结合。允许在城市上堆叠

==============================================================================
网络输入修改为19层
=========================================================
2021.12.03 18:00
最新情况

让网络支持了citytile的选择

夜间出城逻辑改回了yzy写的

cart彻底没了，除非网络输出再加一维。而且其行为逻辑也不太好想

此版本是训练一遍的情况。我正在跑25遍的。
=============================================================================================================================
增加了unit找城市避难的功能，把相关的超参数改成了20到40天（即：允许考虑救援和避难的时间段）


救援改成了多unit救1个city。但救援会出现city之间共用unit的情况,我就先按城市规模大小排序，先救大城市


关于cart仅仅加了 build时0.1的概率为cart，0.9为worker。所以cart现在是不会动的


怀疑有个bug（step out city应该是白天吧），我自行改了，并打上了？？？的注释


现在的问题主要是不用其他数据集的话，没法改变网络的功能，使得cart只能纯规则实现，也完全没考虑道路等级问题


此notebook是把训练次数改为1的结果，暂时还没训25次，那样大约需要1小时。


cart一个可能的逻辑：车为空时出去找worker，找到了就强制要求worker transfer 尤其是把不是木头的资源给cart（因为这些如果被用来build city就浪费了）
车基本满了，就找城市回去。如果cart能用别的方法实现那更好，不行就把这个实现算了。毕竟分数是一方面，咱们的工作如果没涉及cart，最后给分怕是要吃亏。

还有一个观察到的问题，有的city最后会剩余大量资源。这本来是网络应该考虑到的因素，也许应该用规则强制一下。但不知道怎么改。

In [ ]:
import numpy as np
import json
from pathlib import Path
import os
import random
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [ ]:
def seed_everything(seed_value):
    #random will be the same if getting the same seed_value 
    random.seed(seed_value)
    np.random.seed(seed_value)
    #Sets the seed for generating random numbers for the current GPU
    torch.manual_seed(seed_value)
    #set Python Hash Seed to a specific number,making it able to appear again
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    #indicate if cuda is available 
    if torch.cuda.is_available(): 
        #Sets the seed for generating random numbers for the current GPU
        torch.cuda.manual_seed(seed_value)
        #Sets the seed for generating random numbers on all GPUs.
        torch.cuda.manual_seed_all(seed_value)
        #if True, causes cuDNN to only use deterministic convolution algorithms
        torch.backends.cudnn.deterministic = True
        # if True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)

# Preprocessing

In [ ]:
def to_label(action):
    #将action按照空格分成一个长为3的字符串列表
    strs = action.split(' ')
    unit_id = strs[1]
    
    if strs[0]=="r":
        return 0,0,1,0,int(strs[1]),int(strs[2])
    if strs[0]=="bw":
        return 0,0,1,1,int(strs[1]),int(strs[2])
    
    #m代表move，说明是一个可动单位
    if strs[0] == 'm':
        #north、south、west、east
        label = {'c': None, 'n': 0, 's': 1, 'w': 2, 'e': 3}[strs[2]]
    elif strs[0] == 'bcity':
        label = 4
    else:
        label = None
    #return unit_id, label, is_citytile, buildworker, x, y
    return unit_id, label, 0, 0, 0, 0

#判断资源是否所有单位全部耗尽的函数（即一方全部消失在黑暗中，游戏结束）
def depleted_resources(obs):
    for u in obs['updates']:
        if u.split(' ')[0] == 'r':
            return False
    return True


def create_dataset_from_json(episode_dir, team_name='Toad Brigade'): 
    obses = {}
    samples = []
    append = samples.append
    #从../input/lux-ai-episodes寻找训练模型，并加入episodes中
    episodes = [path for path in Path(episode_dir).glob('*.json') if 'output' not in path.name]
    #tqdm为一个显示进度条的库，使得处理进度GUI化
    for filepath in tqdm(episodes): 
        with open(filepath) as f:
            #json_load表示从json文件中读取的数据形成的对象
            json_load = json.load(f)

        ep_id = json_load['info']['EpisodeId']
        #返回rewards数组最大值的索引（本质上是选择最后得分更高的player进行学习）
        index = np.argmax([r or 0 for r in json_load['rewards']])
        #仅使用自己team定义的json文件
        if json_load['info']['TeamNames'][index] != team_name:
            continue
        #对一个json文件的每一步进行操作
        for i in range(len(json_load['steps'])-1):
            #如果在该步执行后存在该"Active"状态说明我方存活
            if json_load['steps'][i][index]['status'] == 'ACTIVE':
                #如果该步状态后依然我方active，load下一步状态的所有动作，放入actions中
                actions = json_load['steps'][i+1][index]['action']
                #load当前步骤后的全地图资源状态
                obs = json_load['steps'][i][0]['observation']
                
                if depleted_resources(obs):
                    break
                
                obs['player'] = index
                #将obs重新整合为一个字典
                obs = dict([
                    (k,v) for k,v in obs.items() 
                    if k in ['step', 'updates', 'player', 'width', 'height']
                ])
                obs_id = f'{ep_id}_{i}'
                #obses为所有obs字典的列表集合
                obses[obs_id] = obs
                                
                for action in actions:
                    #label为当前执行动作对应的下标
                    unit_id, label, is_citytile, buildworker, x, y = to_label(action)
                    if label is not None:
                        if (not is_citytile):
                            #如果动作有方向或者待在city中，那么添加到列表samples中，定义的append类似于C的宏定义
                            append((0,0,0,obs_id, unit_id, label))
                            #LuxDataset里面调用is_citytile, x, y, obs_id, unit_id, action = self.samples[idx]
                        else:
                            append((1,x,y,obs_id, 0, 5+buildworker))
                        
#最终得到全地图每一步执行后的包括资源、各player单位的字典集合obses,所有存活单位每一回合的移动samples
    return obses, samples

In [ ]:
episode_dir = '../input/lux-ai-episodes'
obses, samples = create_dataset_from_json(episode_dir)
print('obses:', len(obses), 'samples:', len(samples))

In [ ]:
#将所有sample的最后一个label，即移动方向制作成list
labels = [sample[-1] for sample in samples]
actions = ['north', 'south', 'west', 'east', 'bcity','research','buildworker']
#打印各个方向动作的总次数
for value, count in zip(*np.unique(labels, return_counts=True)):
    print(f'{actions[value]:^7}: {count:>3}')

# Input For Training

In [ ]:
# Input for Neural Network
def make_input(is_citytile, obs, unit_id, x, y):
    #x,y coordiantes
    width, height = obs['width'], obs['height']
    #//为整数除法，相当于逻辑右移一位
    x_shift = (32 - width) // 2
    y_shift = (32 - height) // 2
    cities = {}
    #制作一个三维全为浮点数0的列表
    b = np.zeros((19, 32, 32), dtype=np.float32)
    
    for update in obs['updates']:
        #split方法通过指定分隔符分隔字符串
        strs = update.split(' ')
        input_identifier = strs[0]
        #a player unit which contains all information
        #str[1]为0时表示worker，为1时表示cart
        if input_identifier == 'u':
            x = int(strs[4]) + x_shift
            y = int(strs[5]) + y_shift
            wood = int(strs[7])
            coal = int(strs[8])
            uranium = int(strs[9])
            if (unit_id == strs[3] and not is_citytile):
                # Position and Cargo
                b[:2, x, y] = (
                    1,#在第0层有unit的位置标上1
                    (wood + coal + uranium) / 100#在第1层对应有unit的位置，标记其装载材料的比例
                )
            else:
                # Units
                team = int(strs[2])
                cooldown = float(strs[6])
                idx = 2 + (team - obs['player']) % 2 * 3#结果为2或者5
                #在第2层标记非unit_id的本方单位位置
                #在第5层标记非unit_id的敌方单位位置
                #在第3层标记敌我的cooldown / 6，第4层标记其装载材料的比例
                b[idx, x, y] = 1
                b[3:5,x,y]=(
                    cooldown / 6,
                    (wood + coal + uranium) / 100
                )
                
        elif input_identifier == 'ct':
            # CityTiles
            team = int(strs[1])
            city_id = strs[2]
            idx=0
            x = int(strs[3]) + x_shift
            y = int(strs[4]) + y_shift
            if (is_citytile and x == int(strs[3]) and y == int(strs[4])):
                b[6:8,x,y]=(
                    1,
                    cities[city_id]
                )
            else:
                idx = 8 + (team - obs['player']) % 2 * 2
            #第8层标记本方citytile位置
            #第10层标记敌方citytile位置
            #第9层标记该city在黑夜能持续几天
            #如果恰好是xy处的citytile，就放在6和7层
            b[idx, x, y] = 1
            b[9,x,y]=cities[city_id]
        elif input_identifier == 'r':
            # Resources
            r_type = strs[1]
            x = int(strs[2]) + x_shift
            y = int(strs[3]) + y_shift
            amt = int(float(strs[4]))
            #第12、13、14层分别放wood、coal、uranium的位置极其装载比例
            b[{'wood': 12, 'coal': 13, 'uranium': 14}[r_type], x, y] = amt / 800
        elif input_identifier == 'rp':
            # Research Points
            team = int(strs[1])
            rp = int(strs[2])
            #第15层放我方研究点数
            #第16层放敌方研究点数
            b[15 + (team - obs['player']) % 2, :] = min(rp, 200) / 200
        elif input_identifier == 'c':
            # Cities参数计算
            city_id = strs[2]
            fuel = float(strs[3])
            lightupkeep = float(strs[4])
            cities[city_id] = min(fuel / lightupkeep, 10) / 10
    
    # Day/Night Cycle
    b[11, :] = obs['step'] % 40 / 40
    # Turns
    b[17, :] = obs['step'] / 360
    # Map Size
    b[18, x_shift:32 - x_shift, y_shift:32 - y_shift] = 1

    return b


class LuxDataset(Dataset):
    def __init__(self, obses, samples):
        self.obses = obses
        self.samples = samples
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        is_citytile, x, y, obs_id, unit_id, action = self.samples[idx]
        obs = self.obses[obs_id]
        state = make_input(is_citytile, obs, unit_id, x, y)
        
        return state, action

# Model for Training

In [ ]:
# Neural Network for Lux AI
#class torch.nn.Module是所有网络的基类，所有模型也应该继承这个类。
#此为基础的卷积层模型
class BasicConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        #padding(int or tuple, optional) - 输入的每一条边补充0的层数,这是为了防止卷积核的大小与步长使得一些边缘数据无法被遍历，所以需要进行填充
        #kerner_size(int or tuple) - 卷积核的尺寸
        #输入的尺度是(N, C_in,H,W)，输出尺度（N,C_out,H_out,W_out）
        #这里padding填充这个数量的目的是保证在一次卷积操作后其尺寸与原尺寸保持相同
        self.conv = nn.Conv2d(
            input_dim, output_dim, 
            kernel_size=kernel_size, 
            padding=(kernel_size[0] // 2, kernel_size[1] // 2)
        )
        
        #对小批量(mini-batch)3d数据组成的4d输入进行批标准化(Batch Normalization)操作
        #在每一个小批量（mini-batch）数据中，计算输入各个维度的均值和标准差
        #在训练时，该层计算每次输入的均值与方差，并进行移动平均。移动平均默认的动量值为0.1。
        #输入输出相同
        #BatchNorm2d函数是在使用卷积计算后进行归一化，防止relu前的数据过大导致网络不稳定
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = self.conv(x)
        h = self.bn(h) if self.bn is not None else h
        return h


class LuxNet(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 12, 32
        #model initialization
        self.conv0 = BasicConv2d(19, filters, (3, 3), True)
        #将submodules保存在一个list中。ModuleList可以像一般的Python list一样被索引，即每一个都是用于存储一个layer的
        self.blocks = nn.ModuleList([BasicConv2d(filters, filters, (3, 3), True) for _ in range(layers)])
        #nn.Linear（）是用于设置网络中的全连接层的，需要注意在二维图像处理的任务中
        #全连接层的输入与输出一般都设置为二维张量，形状通常为[batch_size, size]
        #其中前两个参数分别为in的size，7为输出的size
        self.head_p = nn.Linear(filters, 7, bias=False)

    #神经网络的前向传播
    def forward(self, x):
        #此为激励层的内容，relu_是一种非线性激活函数，本质上在深度神经网络上可以逼近任何类型的函数
        #relu函数的第二个参数inplace默认值为false，表示不改变输入的数据
        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))
        #view函数相当于resize的功能，将原来的tensor变换成指定的维度
        #这里是将激励值乘入输入x，再将其重新张成h.size(0)*h.size(1)*剩余的形式并求和
        h_head = (h * x[:,:1]).view(h.size(0), h.size(1), -1).sum(-1)
        #调用head_p函数张成二维向量
        #上面说的不太对，view成二维(h.size(0),filters)，再通过全连接层head_p，规模变为(h.size(0),7)
        p = self.head_p(h_head)
        return p

# Train

In [ ]:
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):
    best_acc = 0.0
    #动态改变学习率的sheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1)
    
    for epoch in range(num_epochs):
        #显式地指定model使用gpu
        model.cuda()
        #在Pytorch训练中有两种模式，train为训练模式，eval为评估模式
        for phase in ['train', 'val']:
            if phase == 'train':
                #在使用pytorch构建神经网络的时候，训练过程中会在程序上方添加一句model.train()，作用是启用batch normalization和dropout
                model.train()
            else:
                #测试过程中会使用model.eval()，这时神经网络会沿用batch normalization的值，并不使用dropout，即我们这里不是在训练模型
                #那么就不需要进行dropout改变模型，故调用该函数保证dropout不变
                model.eval()
                
            epoch_loss = 0.0
            epoch_acc = 0
            
            dataloader = dataloaders_dict[phase]
            #对模型进行多次训练
            for item in tqdm(dataloader, leave=False):
                states = item[0].cuda().float()
                actions = item[1].cuda().long()
                #将模型的参数梯度初始化为0
                optimizer.zero_grad()
                #在我们处于训练模式下时，允许计算局部梯度
                with torch.set_grad_enabled(phase == 'train'):
                    policy = model.forward(states)
                    #criterion为损失函数，计算损失
                    loss = criterion(policy, actions)
                    #返回所有张量最大值
                    _, preds = torch.max(policy, 1)

                    #如果处于训练模式下，损失函数backward进行反向传播梯度的计算，并使用优化器的step函数来更新参数
                    if phase == 'train':
                        #当前Variable对leaf variable求偏导，计算好梯度
                        loss.backward()
                        #根据前面求出的梯度更新参数
                        optimizer.step()

                    epoch_loss += loss.item() * len(policy)
                    epoch_acc += torch.sum(preds == actions.data)
            #归一化
            data_size = len(dataloader.dataset)
            epoch_loss = epoch_loss / data_size
            epoch_acc = epoch_acc.double() / data_size
            if phase == 'train':
                scheduler.step()
            print(f'Epoch {epoch + 1}/{num_epochs} | {phase:^5} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')
        #产生了更好结果的情况
        if epoch_acc > best_acc:
            traced = torch.jit.trace(model.cpu(), torch.rand(1, 19, 32, 32))
            traced.save('model.pth')
            best_acc = epoch_acc

In [ ]:
model = LuxNet()
#train_test_split将原始数据按照比例划分为“测试集”“训练集”
# test_size：样本占比，如果是整数的话就是样本的数量

# random_state：是随机数的种子。
# 随机数种子：其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。
#stratify是为了保持split前类的分布，保证其在训练集中的分布比例不变
train, val = train_test_split(samples, test_size=0.15, random_state=42, stratify=labels)
batch_size = 64
#shuffle为false表示不打乱传入的数据
#num_workers表示了会有多少进程共同执行
train_loader = DataLoader(
    LuxDataset(obses, train), 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=2
)
val_loader = DataLoader(
    LuxDataset(obses, val), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=2
)
dataloaders_dict = {"train": train_loader, "val": val_loader}
#交叉熵损失函数，一种使用对数的损失函数
criterion = nn.CrossEntropyLoss()
#利用Adam(Adaptive Moment Estimation)进行优化
#它的优点主要在于经过偏置校正后，每一次迭代学习率都有个确定范围，使得参数比较平稳
#lr为学习率，过高会导致不稳定与过拟合
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=25)

# Submission

In [ ]:
from kaggle_environments import make

env = make("lux_ai_2021", configuration={"width": 24, "height": 24, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run(['agent.py', 'agent.py'])
env.render(mode="ipython", width=1200, height=800)

In [ ]:
!tar -czf submission.tar.gz *